### IMPORTACIONES

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import re

### EXPLORAMOS EL DATA

In [52]:
df_pc = pd.read_csv('./data/train.csv')

In [53]:
df_pc.head()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [54]:
df_pc.info()
# no tengo nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [55]:
x_train = df_pc#.drop('Price_euros', axis= 1)
# y_train = df_pc['Price_euros']

In [56]:
for feature in x_train.columns:

    cardinalidad = x_train[feature].nunique()
    porc_cardinalidad = cardinalidad / len(x_train)

    print(feature.upper())
    print('-'*10)
    print(f'Cardinalidad feature "{feature}" -> {cardinalidad}.')
    print(f'Cardinalidad tanto por 1 feature "{feature}" -> {porc_cardinalidad}.\n')

ID
----------
Cardinalidad feature "id" -> 912.
Cardinalidad tanto por 1 feature "id" -> 1.0.

LAPTOP_ID
----------
Cardinalidad feature "laptop_ID" -> 912.
Cardinalidad tanto por 1 feature "laptop_ID" -> 1.0.

COMPANY
----------
Cardinalidad feature "Company" -> 19.
Cardinalidad tanto por 1 feature "Company" -> 0.020833333333333332.

PRODUCT
----------
Cardinalidad feature "Product" -> 475.
Cardinalidad tanto por 1 feature "Product" -> 0.5208333333333334.

TYPENAME
----------
Cardinalidad feature "TypeName" -> 6.
Cardinalidad tanto por 1 feature "TypeName" -> 0.006578947368421052.

INCHES
----------
Cardinalidad feature "Inches" -> 15.
Cardinalidad tanto por 1 feature "Inches" -> 0.01644736842105263.

SCREENRESOLUTION
----------
Cardinalidad feature "ScreenResolution" -> 35.
Cardinalidad tanto por 1 feature "ScreenResolution" -> 0.03837719298245614.

CPU
----------
Cardinalidad feature "Cpu" -> 104.
Cardinalidad tanto por 1 feature "Cpu" -> 0.11403508771929824.

RAM
----------
Cardina

In [57]:
'''
ID. como feature no aporta nada imagino, solo la necesito para identificar el pc, lo puedo poner como indice
LAPTOP_ID. me sobra
COMPANY. categorica
PRODUCT. numerica continua, quiero estudiar las variables, habra repetidas o similares?
TYPENAME. categorica
INCHES. numerica discreta
SCREENRESOLUTION. categorica, discreta. igual puedo simplificar esto?
CPU. continua, estudiar si puedo simplificarla, en pllan por el procesador por ejemplo
RAM. pasar a numerica discreta
MEMORY. igual puedo poner la memoria como numerica en gb y crear una columna anexa que sea SDD o HDD en binaria?
GPU. continua, no se bien que hacer con ella
OPSYS. categorica
WEIGHT. numerica continua. poner en kg o gramos?
'''

'\nID. como feature no aporta nada imagino, solo la necesito para identificar el pc, lo puedo poner como indice\nLAPTOP_ID. me sobra\nCOMPANY. categorica\nPRODUCT. numerica continua, quiero estudiar las variables, habra repetidas o similares?\nTYPENAME. categorica\nINCHES. numerica discreta\nSCREENRESOLUTION. categorica, discreta. igual puedo simplificar esto?\nCPU. continua, estudiar si puedo simplificarla, en pllan por el procesador por ejemplo\nRAM. pasar a numerica discreta\nMEMORY. igual puedo poner la memoria como numerica en gb y crear una columna anexa que sea SDD o HDD en binaria?\nGPU. continua, no se bien que hacer con ella\nOPSYS. categorica\nWEIGHT. numerica continua. poner en kg o gramos?\n'

In [58]:
x_train.drop('laptop_ID', axis= 1, inplace= True)
# eliminamos la columna de laptop_id

In [59]:
x_train.set_index('id', inplace= True)
# ponemos id como indice del df

## vamos a estudiar las variables una por una a ver como podemos optimizarlas  

variable company

In [60]:
x_train['Company'].unique()

array(['Dell', 'Asus', 'Toshiba', 'MSI', 'HP', 'Lenovo', 'Acer', 'Apple',
       'Fujitsu', 'Mediacom', 'Razer', 'Samsung', 'LG', 'Microsoft',
       'Vero', 'Google', 'Huawei', 'Xiaomi', 'Chuwi'], dtype=object)

In [61]:
x_train['Company'].value_counts()

Company
Lenovo       208
Dell         207
HP           189
Asus         108
Acer          75
Toshiba       37
MSI           34
Apple         17
Mediacom       7
Razer          6
Samsung        6
Fujitsu        3
Microsoft      3
Vero           3
Xiaomi         3
Google         2
Chuwi          2
LG             1
Huawei         1
Name: count, dtype: int64

In [62]:
ordinal_company = OrdinalEncoder()

x_train['Company'] = ordinal_company.fit_transform(x_train[['Company']])
# a compañia le pasamos el ordinal y convertimos a numerica

variable producto

In [63]:
x_train['Product'].nunique()

475

In [64]:
x_train['Product'].unique()

array(['Inspiron 5567', 'X541NA (N4200/4GB/1TB/W10)', 'Portege Z30-C-1CW',
       'Alienware 17', 'Latitude E7270', 'ZenBook Flip',
       'X541UA-DM1897 (i3-6006U/4GB/256GB/FHD/Linux)', 'GP62M 7RDX',
       'UX430UQ-GV209R (i7-7500U/8GB/256GB/GeForce', 'Rog GL553VE-FY052T',
       'X540SA-RBPDN09 (N3710/4GB/1TB/W10)', 'Inspiron 5570',
       'Tecra A50-C-1ZV', '15-cd005nv (A9-9420/6GB/256GB/Radeon',
       'Satellite Pro', 'Latitude 5590', 'Yoga 520-14IKB',
       'Aspire E5-575', 'Thinkpad P51', 'Aspire A517-51G', 'Yoga Book',
       '17-ak002nv (A10-9620P/6GB/2TB/Radeon', 'EliteBook 820',
       'Omen 17-W295', 'Precision M5520',
       'X540UA-DM186 (i3-6006U/4GB/1TB/FHD/Linux)', 'Extensa EX2540-58KR',
       'IdeaPad 310-15IKB', 'IdeaPad 320-15IAP', 'ProBook 640',
       'Inspiron 3576', 'GL553VE-FY082T (i7-7700HQ/8GB/1TB',
       'B51-80 (i7-6500U/8GB/1008GB/Radeon', 'Vostro 3568',
       'Chromebook CB5-571-C1DZ', 'IdeaPad 320-15ABR', 'Inspiron 3552',
       'X553SA-XX021T (N305

In [65]:
x_train['Product'].value_counts()

Product
Inspiron 3567                                21
XPS 13                                       20
250 G6                                       19
Vostro 3568                                  16
Inspiron 5570                                11
                                             ..
Elitebook Folio                               1
Q304UA-BHI5T11 (i5-7200U/6GB/1TB/FHD/W10)     1
ThinkPad T470p                                1
V310-15IKB (i5-7200U/4GB/1TB/No               1
Portege Z30-C-16H                             1
Name: count, Length: 475, dtype: int64

In [66]:
ordinal_product = OrdinalEncoder()

x_train['Product'] = ordinal_product.fit_transform(x_train[['Product']])
# a product le pasamos el ordinal y convertimos a numerica

variable tipename

In [67]:
x_train['TypeName'].unique()

array(['Notebook', 'Gaming', 'Ultrabook', '2 in 1 Convertible', 'Netbook',
       'Workstation'], dtype=object)

In [68]:
ordinal_typename = OrdinalEncoder()

x_train['TypeName'] = ordinal_typename.fit_transform(x_train[['TypeName']])

variable inches

In [69]:
x_train['Inches'].unique()

# a esta columna no vamos a hacerle nada

array([15.6, 13.3, 12.5, 14. , 17.3, 10.1, 11.6, 15.4, 15. , 12. , 13.5,
       12.3, 13. , 13.9, 18.4])

varaible screenresolution

In [70]:
x_train['ScreenResolution'].unique()

array(['1366x768', 'Full HD 1920x1080', 'IPS Panel 4K Ultra HD 3840x2160',
       'Full HD / Touchscreen 1920x1080', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'IPS Panel Touchscreen 1920x1200',
       '4K Ultra HD / Touchscreen 3840x2160', 'Touchscreen 1366x768',
       'IPS Panel Retina Display 2880x1800', 'IPS Panel 1366x768',
       'IPS Panel Quad HD+ / Touchscreen 3200x1800',
       'Quad HD+ / Touchscreen 3200x1800', '4K Ultra HD 3840x2160',
       'IPS Panel 2560x1440', '1600x900', 'Touchscreen 2560x1440',
       'IPS Panel Quad HD+ 2560x1440',
       'IPS Panel Retina Display 2304x1440', '1440x900',
       'IPS Panel Retina Display 2560x1600', 'Touchscreen 2256x1504',
       '2560x1440', 'IPS Panel Touchscreen / 4K Ultra HD 3840x2160',
       'IPS Panel Touchscreen 2560x1440', '1920x1080',
       'Touchscreen 2400x1600', 'IPS Panel Full HD 2160x1440',
       'Quad HD+ 3200x1800',
       'IPS Panel 4K Ultra HD / Touchscreen 3840x21

In [71]:
# vamos a utilizar expresiones regulares para extraer la resolucion en plan 1234x1234
# patron = r'\d{3,4}x\d{3,4}'

In [72]:
# creamos una funcion para pasarla por el df

def resolution_converter(texto):

    patron = r'\d{3,4}x\d{3,4}'
    res = re.findall(patron, texto)

    return res[0]

In [73]:
x_train['ScreenResolution'] = x_train['ScreenResolution'].apply(resolution_converter)

In [74]:
x_train['ScreenResolution'].value_counts()
# me devuelve 14 categorias, me vale

ScreenResolution
1920x1080    582
1366x768     229
3840x2160     34
3200x1800     15
2560x1440     14
1600x900      11
2304x1440      6
1920x1200      5
2560x1600      5
2880x1800      3
2256x1504      3
1440x900       2
2400x1600      2
2160x1440      1
Name: count, dtype: int64

In [75]:
ordinal_screen = OrdinalEncoder()

x_train['ScreenResolution'] = ordinal_screen.fit_transform(x_train[['ScreenResolution']])

variable cpu

In [76]:
x_train['Cpu'].unique()

# aqui me parece a mi que vamos a utilizar tambien las expresiones regulares para extraer los GHz

array(['Intel Core i7 7500U 2.7GHz',
       'Intel Pentium Quad Core N4200 1.1GHz',
       'Intel Core i5 6200U 2.3GHz', 'Intel Core i7 7700HQ 2.8GHz',
       'Intel Core i5 6300U 2.4GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Pentium Quad Core N3710 1.6GHz',
       'Intel Core i7 8550U 1.8GHz', 'AMD A9-Series A9-9420 3GHz',
       'Intel Pentium Dual Core 4405U 2.1GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 7100U 2.4GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Xeon E3-1535M v6 3.1GHz',
       'Intel Atom x5-Z8550 1.44GHz', 'AMD A10-Series 9620P 2.5GHz',
       'Intel Core i5 7300U 2.6GHz',
       'Intel Celeron Dual Core N3350 1.1GHz',
       'Intel Core i7 6500U 2.5GHz', 'Intel Core i3 6006U 2.0GHz',
       'Intel Celeron Dual Core 3205U 1.5GHz',
       'AMD A12-Series 9720P 3.6GHz',
       'Intel Celeron Dual Core N3050 1.6GHz',
       'Intel Core i3 7130U 2.7GHz', 'Intel Core i5 7300HQ 2.5GHz',
       'Intel Core i5 7200U 2.7GHz', 'Intel Core i7 7820HQ 2.9GHz

In [77]:
# creamos una funcion para pasarla por el df

def cpu_converter(texto):

    patron = r'\d{1,2}\.\d{1,2}GHz'
    res = re.findall(patron, texto)

    if res == []:

        patron = r'\dGHz'
        res = re.findall(patron, texto)
    
    res[0] = res[0].replace('GHz', '')

    return float(res[0])

In [78]:
x_train['Cpu'] = x_train['Cpu'].apply(cpu_converter)

In [79]:
x_train['Cpu'].value_counts()

Cpu
2.50    208
2.80    114
2.70    107
1.60     98
2.30     65
2.00     58
1.80     48
2.60     47
1.10     40
2.40     40
2.90     15
3.00     13
1.44     11
1.20     10
1.50      8
2.20      8
1.30      6
3.60      4
3.10      3
2.10      3
0.90      2
1.00      1
3.20      1
1.90      1
1.92      1
Name: count, dtype: int64

variable ram

In [80]:
# vamos a extraerle el valor numerico con expresiones regulares y hacerla numerica

x_train['Ram'].unique()

array(['8GB', '4GB', '16GB', '12GB', '6GB', '32GB', '2GB', '24GB'],
      dtype=object)

In [81]:
# creamos una funcion para pasarla por el df

def ram_converter(texto):

    patron = r'\d{1,2}'
    res = re.findall(patron, texto)

    return int(res[0])

In [82]:
x_train['Ram'] = x_train['Ram'].apply(ram_converter)

variable memory

In [83]:
x_train['Memory'].unique()

array(['1TB HDD', '256GB SSD', '256GB SSD +  1TB HDD',
       '512GB SSD +  2TB HDD', '128GB SSD +  1TB HDD', '128GB SSD',
       '1TB SSD', '64GB Flash Storage', '2TB HDD', '500GB HDD',
       '1.0TB Hybrid', '16GB Flash Storage', '256GB Flash Storage',
       '180GB SSD', '32GB Flash Storage', '512GB SSD',
       '512GB SSD +  1TB HDD', '256GB SSD +  500GB HDD',
       '256GB SSD +  2TB HDD', '1TB SSD +  1TB HDD', '64GB SSD',
       '128GB Flash Storage', '256GB SSD +  1.0TB Hybrid', '32GB SSD',
       '512GB Flash Storage', '240GB SSD', '256GB SSD +  256GB SSD',
       '16GB SSD', '8GB SSD', '64GB Flash Storage +  1TB HDD',
       '1.0TB HDD', '1TB HDD +  1TB HDD', '512GB SSD +  512GB SSD',
       '128GB SSD +  2TB HDD', '32GB HDD', '512GB SSD +  1.0TB Hybrid'],
      dtype=object)

In [84]:
x_train['Memory'].nunique() # creo que la voy a dejar igual, no voy a modificarla

36

In [85]:
ordinal_memory = OrdinalEncoder()

x_train['Memory'] = ordinal_memory.fit_transform(x_train[['Memory']])

variable gpu

In [86]:
x_train['Gpu'].unique()

array(['AMD Radeon R7 M445', 'Intel HD Graphics 505',
       'Intel HD Graphics 520', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GT 940MX', 'Nvidia GeForce GTX 1050',
       'Nvidia GeForce 940MX', 'Nvidia GeForce GTX 1050 Ti',
       'Intel HD Graphics 405', 'Intel UHD Graphics 620',
       'AMD Radeon 530', 'Intel HD Graphics 510', 'Intel HD Graphics 620',
       'Nvidia Quadro M2200M', 'Nvidia GeForce MX150',
       'Intel HD Graphics 400', 'Nvidia Quadro M1200',
       'Nvidia GeForce 920MX', 'Intel HD Graphics 500', 'AMD Radeon 520',
       'AMD Radeon R5 M330', 'AMD Radeon R5 M420X', 'Intel HD Graphics',
       'Intel HD Graphics 630', 'AMD Radeon R5 M430',
       'Intel Iris Pro Graphics', 'Nvidia GeForce GTX 940MX',
       'Intel HD Graphics 515', 'Nvidia GeForce GTX 1080',
       'AMD Radeon R2 Graphics', 'Nvidia GeForce 150MX', 'AMD Radeon R5',
       'Nvidia GeForce GTX 1050Ti', 'AMD Radeon R9 M385',
       'Nvidia GeForce 930MX', 'Nvidia GeForce GTX 1060',
       'Nv

In [87]:
x_train['Gpu'].nunique()

# creo que voy a categorizarlas por la marca, amd, intel, nvidia e intel

91

In [88]:
# creamos una funcion para pasarla por el df

def gpu_converter(texto):

    patrones = [r'AMD', r'Nvidia', r'Intel']

    for i in patrones:
        res = re.findall(i, texto)
        if res != []:
            break
    try:
        return res[0]
    except:
        return 'Others'

In [89]:
x_train['Gpu'] = x_train['Gpu'].apply(gpu_converter)

In [90]:
ordinal_gpu = OrdinalEncoder()

x_train['Gpu'] = ordinal_gpu.fit_transform(x_train[['Gpu']])

variable opsys

In [91]:
x_train['OpSys'].unique()

# este se va a quedar asi, en ordinal

array(['Windows 10', 'Windows 7', 'Linux', 'Chrome OS', 'No OS',
       'Mac OS X', 'Windows 10 S', 'macOS', 'Android'], dtype=object)

In [92]:
ordinal_opsys = OrdinalEncoder()

x_train['OpSys'] = ordinal_gpu.fit_transform(x_train[['OpSys']])

variable weight

In [93]:
x_train['Weight'].unique()

# aqui vamos a hacerla numerica extrayendo los pesos

array(['2.36kg', '2kg', '1.2kg', '4.42kg', '1.26kg', '2.26kg', '2.4kg',
       '1.3kg', '2.5kg', '2.65kg', '2.02kg', '2.2kg', '1.95kg', '1.75kg',
       '1.88kg', '1.74kg', '2.23kg', '3kg', '0.69kg', '2.71kg', '3.35kg',
       '1.78kg', '2.13kg', '2.32kg', '2.18kg', '2.20kg', '2.1kg', '1.7kg',
       '2.3kg', '1.91kg', '2.31kg', '1.4kg', '1.48kg', '2.04kg', '1.54kg',
       '1.86kg', '1.98kg', '1.47kg', '2.67kg', '1.85kg', '1.5kg',
       '0.97kg', '1.6kg', '1.68kg', '2.09kg', '2.33kg', '3.49kg',
       '3.78kg', '1.36kg', '2.77kg', '1.29kg', '1.62kg', '1.05kg',
       '2.05kg', '1.9kg', '1.28kg', '1.49kg', '2.62kg', '2.0kg', '1.63kg',
       '1.37kg', '2.73kg', '2.69kg', '1.25kg', '4.14kg', '2.9kg',
       '1.23kg', '2.37kg', '1.10kg', '1.90kg', '2.63kg', '1.39kg',
       '2.8kg', '2.94kg', '1.43kg', '1.22kg', '2.21kg', '1.32kg', '2.6kg',
       '1.45kg', '1.14kg', '4.36kg', '1.8kg', '1.1kg', '3.3kg', '3.2kg',
       '4.2kg', '1.35kg', '2.19kg', '1.96kg', '4.0kg', '1.84kg', '1.65kg',


In [94]:
def weight_converter(texto):

    patron = r'\d{1,2}\.\d{1,2}'
    res = re.findall(patron, texto)

    if res == []:

        patron = r'\d'
        res = re.findall(patron, texto)

    return float(res[0])

In [95]:
x_train['Weight'] = x_train['Weight'].apply(weight_converter)

## final

In [96]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 268 to 1179
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    float64
 1   Product           912 non-null    float64
 2   TypeName          912 non-null    float64
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    float64
 5   Cpu               912 non-null    float64
 6   Ram               912 non-null    int64  
 7   Memory            912 non-null    float64
 8   Gpu               912 non-null    float64
 9   OpSys             912 non-null    float64
 10  Weight            912 non-null    float64
 11  Price_euros       912 non-null    float64
dtypes: float64(11), int64(1)
memory usage: 92.6 KB


In [97]:
x_train.describe()

# esto vamos a tener que escalarlo

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
count,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000,912.000000
mean,6.449561,234.040570,2.589912,15.011404,2.962719,2.287895,8.188596,15.289474,1.160088,4.796053,2.032105,1108.122873
std,4.146293,133.403803,1.232841,1.411744,2.929230,0.513277,4.899827,7.729197,0.634385,1.075659,0.653780,714.597741
min,0.000000,0.000000,0.000000,10.100000,0.000000,0.900000,2.000000,0.000000,0.000000,0.000000,0.690000,174.000000
25%,4.000000,136.750000,2.000000,14.000000,0.000000,2.000000,4.000000,9.000000,1.000000,5.000000,1.500000,589.000000
50%,7.000000,218.500000,3.000000,15.600000,3.000000,2.500000,8.000000,15.000000,1.000000,5.000000,2.040000,949.000000
75%,10.000000,347.250000,3.000000,15.600000,3.000000,2.700000,8.000000,22.000000,2.000000,5.000000,2.300000,1458.500000
max,18.000000,474.000000,5.000000,18.400000,13.000000,3.600000,32.000000,35.000000,2.000000,8.000000,4.600000,6099.000000


In [98]:
for i in x_train.columns:
    print(i, x_train[i].nunique())

Company 19
Product 475
TypeName 6
Inches 15
ScreenResolution 14
Cpu 25
Ram 8
Memory 36
Gpu 3
OpSys 9
Weight 150
Price_euros 614


In [99]:
x_train.to_csv('./data/train_limpio.csv', index=False)

In [100]:
x_test = pd.read_csv('./data/test.csv')

# variable ladtop id

x_test.drop('laptop_ID', axis= 1, inplace= True)

# variable company

x_test['Company'] = ordinal_company.transform(x_test[['Company']])

# variable product

x_test['Product'] = ordinal_product.transform(x_test[['Product']])

# variable typename


x_test['TypeName'] = ordinal_typename.transform(x_test[['TypeName']])

# variable inches

# nada

# variable screenresolution

def resolution_converter(texto):

    patron = r'\d{3,4}x\d{3,4}'
    res = re.findall(patron, texto)

    return res[0]

x_test['ScreenResolution'] = x_test['ScreenResolution'].apply(resolution_converter)

ordinal_screen = OrdinalEncoder()

x_test['ScreenResolution'] = ordinal_screen.transform(x_test[['ScreenResolution']])

# variable cpu

def cpu_converter(texto):

    patron = r'\d{1,2}\.\d{1,2}GHz'
    res = re.findall(patron, texto)

    if res == []:

        patron = r'\dGHz'
        res = re.findall(patron, texto)
    
    res[0] = res[0].replace('GHz', '')

    return float(res[0])

x_test['Cpu'] = x_test['Cpu'].apply(cpu_converter)

# variable ram

def ram_converter(texto):

    patron = r'\d{1,2}'
    res = re.findall(patron, texto)

    return int(res[0])

x_test['Ram'] = x_test['Ram'].apply(ram_converter)

# variable memory


x_test['Memory'] = ordinal_memory.transform(x_test[['Memory']])

# variable gpu

def gpu_converter(texto):

    patrones = [r'AMD', r'Nvidia', r'Intel']

    for i in patrones:
        res = re.findall(i, texto)
        if res != []:
            break

    try:
        return res[0]
    except:
        return 'Others'

x_test['Gpu'] = x_test['Gpu'].apply(gpu_converter)


x_test['Gpu'] = ordinal_gpu.transform(x_test[['Gpu']])

# variable opsys


x_test['OpSys'] = ordinal_gpu.transform(x_test[['OpSys']])

# variable weight

def weight_converter(texto):

    patron = r'\d{1,2}\.\d{1,2}'
    res = re.findall(patron, texto)

    if res == []:

        patron = r'\d'
        res = re.findall(patron, texto)

    return float(res[0])

x_test['Weight'] = x_test['Weight'].apply(weight_converter)

x_test.to_csv('./data/test_limpio.csv', index=False)

ValueError: Found unknown categories ['Yoga 11e', 'Precision 7720', 'Inspiron 7779', 'ES1-523-84K7 (A8-7410/8GB/256GB/FHD/W10)', 'LapBook 12.3', 'V330-15IKB (i3-7130U/4GB/128GB/FHD/W10)', 'F756UX-T4201D (i7-7500U/8GB/128GB', '15-BW004nv (A9-9420/4GB/256GB/Radeon', 'Insprion 5767', 'Rog G752VY-GC229T', 'FX753VD-GC007T (i7-7700HQ/8GB/1TB', 'Rog GL753VE-GC070T', 'X550VX-XX015D (i5-6300HQ/4GB/1TB/GeForce', 'Ideapad 520-15IKBR', 'Lenovo IdeaPad', 'Q524UQ-BHI7T15 (i7-7500U/12GB/2TB/GeForce', 'ZenBook UX310UA-FB485T', '110-15ACL (A6-7310/4GB/500GB/W10)', 'V131 (X5-Z8350/4GB/32GB/FHD/W10)', '15-BW094nd (A6-9220/8GB/128GB/W10)', 'Portege Z30-C-16P', '15-BS028nv (i3-6006U/4GB/1TB/Radeon', 'B51-80 (i5-6200U/8GB/1008GB/Radeon', 'Thinkpad T470s', 'Tecra X40-D-10Z', '15-bs023nv (i3-6006U/4GB/1TB/FHD/W10)', 'FX753VD-GC461T (i7-7700HQ/16GB/1TB', 'VivoBook L402NA', 'Inspiron 5378', 'Pavilion 14-BK001nv', 'FX502VM-DM560T (i7-7700HQ/8GB/1TB', 'ThinkPad L460', 'GE72VR 6RF', '17-Y002nv (A10-9600P/6GB/2TB/R